Establish connection

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


Insert query here

In [2]:
%%bigquery param1 --project athena-179008
SELECT 
	a.orderId
	, a.partner
	, CASE WHEN a.status<>'Approved' then 'Not Approved' else a.status end as application_status
	, a.cli_status as paylater_status
	, CASE COALESCE((active_trx+finished_trx),0) WHEN 0 THEN 'Never Use Paylater' ELSE 'Ever Use Paylater' END paylater_use_status
	, COALESCE((active_trx+finished_trx),0) as total_trx
	, app.applicantPersonalEmail as email
    , app.applicantMobilePhoneNumber1 as phone_number
    , upper(app.applicantName) as name
    , SUBSTR(app.applicantGender,1,1) as gender
    , app.currentCompanyAddressCity as company_address_city
    , app.currentCompanyAddressProvince as company_address_province
    , app.currentCompanyAddressPostalCode as company_address_zipcode
    , 'ID' as company_address_country
    , app.applicantCurrentResidenceCity as address_city
    , app.applicantCurrentResidenceProvince as address_province
    , app.applicantCurrentResidencePostalCode as address_zipcode
    , 'ID' as address_country
    , date(app.applicantDateOfBirth) as date_of_birth
    , extract(year from app.applicantDateOfBirth) as year_of_birth
    , DATE_DIFF(current_date,date(app.applicantDateOfBirth), YEAR) - IF(EXTRACT(MONTH FROM date(app.applicantDateOfBirth))*100 + EXTRACT(DAY FROM date(app.applicantDateOfBirth)) > EXTRACT(MONTH FROM current_date)*100 + EXTRACT(DAY FROM current_date),1,0)  as age
FROM athena-179008.vayu_data_mart.indodanamarketing_cli_application a 
INNER JOIN data-platform-indodana.vayu.indodana_athena_applications app ON app.orderId=a.orderId
LEFT JOIN athena-179008.vayu_data_mart.view_cli_contracts c 
	ON a.orderId=c.orderId 
	AND (active_trx+finished_trx)>0
WHERE a.partner IN ('BLIBLI','TIKET')

Sample 5 rows to check

In [4]:
param1.shape
# param1.dtypes

(975016, 21)

In [5]:
param1.head()

,orderId,partner,application_status,paylater_status,paylater_use_status,total_trx,email,phone_number,name,gender,...,company_address_province,company_address_zipcode,company_address_country,address_city,address_province,address_zipcode,address_country,date_of_birth,year_of_birth,age
0,CLI-QTSWGHRK,TIKET,Approved,ACTIVE,Ever Use Paylater,1,ihsan71nisa81@gmail.com,+6285277972196,KHAIRUNNISA,F,...,KEPULAUAN RIAU,None,ID,KOTA BATAM,KEPULAUAN RIAU,29426,ID,1995-05-18,1995.0,26.0
1,CLI-8T519N5M,TIKET,Not Approved,None,Never Use Paylater,0,shofiafawaid@gmail.com,+6287785658141,SHOFIYAH FAUZI,F,...,JAWA TIMUR,None,ID,KABUPATEN BANGKALAN,JAWA TIMUR,69162,ID,1999-03-25,1999.0,22.0
2,CLI-D5AUPNV4,BLIBLI,Not Approved,None,Never Use Paylater,0,grestaphillea@yahoo.co.id,+6282228831989,GRESTA PHILLEA HANDONO,None,...,JAWA TENGAH,59333,ID,KABUPATEN KUDUS,JAWA TENGAH,59314,ID,None,NaN,NaN
3,CLI-ZZVTCRYG,BLIBLI,Not Approved,None,Never Use Paylater,0,ahraidaa@gmail.com,+6281318164500,AHRAIDA AFRIANTI,F,...,BANTEN,15325,ID,KOTA JAKARTA SELATAN,DKI JAKARTA,12330,ID,1988-04-09,1988.0,33.0
4,CLI-E3TPM1XZ,TIKET,Not Approved,None,Never Use Paylater,0,ambarwatishb@gmail.com,+6282130110011,AMBAR WATI,F,...,DKI JAKARTA,10110,ID,KOTA JAKARTA TIMUR,DKI JAKARTA,13790,ID,1996-04-11,1996.0,25.0


Export to excel/csv

In [6]:
from datetime import date
# print(date.today())
today_str = str(date.today()).replace('-','')
filename = 'Marketing_Audience_Tiket_Blibli_' + today_str + '.xlsx'
print(filename)

Marketing_Audience_Tiket_Blibli_20210901.xlsx


In [ ]:
param1.to_excel(filename,index=False)